In [1]:
import pandas as pd
import numpy as np
import re

In [8]:
with open('dim_customer.sql') as f:
    lines = f.readlines()
    f.seek(0)
    text = f.read()

Hundred characters

In [3]:
def hundred_chars(lines):
    print('Lines of SQL should be no longer than ~100 characters. Please use new line:')
    for count, i in enumerate(lines):
        cnt = 0
        for j in i:
            if j != ' ':
                cnt=cnt+1
        if cnt>100:
            print("\nLine no. {}\n{}".format(count+1,i.strip()))
    print('\n--------------------------------\n')

Comma at the end of the line

In [4]:
def comma_at_end(lines):
    lines_sign = []
    print("For following lines comma is present at start of the line: \n")
    for count, i in enumerate(lines):
        if i.lstrip() != '':  
            if i.lstrip()[0].__contains__(","):
                lines_sign.append([count,i])
                print('Line no. {}\n{}'.format(count,i.strip()))
    print('\n--------------------------------\n')

Inner Join instead of Join keyword 

In [5]:
def inner_join_keyword(lines):
    lines_join = []
    print('Inner keyword not present before join in following lines:\n')
    for count, i in enumerate(lines):
        if i.__contains__("join"):
            lines_join.append([count,i])

    lines_after_join = []
    flag = 0
    for count, i in enumerate(lines):
        if flag == 1:
            flag = 0
            lines_after_join.append([count, i])
        if i.__contains__("join"):
            flag = 1

    join_ls = ['inner', 'left', 'outer', 'right']
    for count, i in enumerate(lines_join):
        flag = 0
        for k in join_ls:
            if flag == 0:
                if k in i[1]:
                    flag = 1
        if flag ==0:
            print("Line no. {}\n{}".format(i[0]+1,i[1].strip()))
    print('\n--------------------------------\n')
    print("Use 'On' instead of 'Using' in following lines:\n")
    for count, i in enumerate(lines_after_join):
        if ' using' in i[1]:
            print(f"Line no. {i[0]+1}\n{i[1].strip()}")
    print('\n--------------------------------\n')

Not equals as !=

In [6]:
def not_equals_sign(lines):
    print('Use != instead of <> in following lines: \n')
    lines_sign = []
    for count, i in enumerate(lines):
        if i.__contains__("<>"):
            lines_sign.append([count,i])
    for count, i in enumerate(lines_sign):
        print("Line no. {}\n{}".format(i[0]+1,i[1].strip()))
    print('\n--------------------------------\n')

Check snake casing

In [1]:
def snake_casing(lines):
    lines_uppercase = []
    flag=0
    reg = '[A-Za-z]+.'
    print('Following lines contain uppercase characters:\n')
    for count, i in enumerate(lines):
        if i.islower()==False and i!='\n':
            x=i.split("'")
            for cnt,j in enumerate(x):
                if re.search(reg, j):
                    if cnt%2 ==0 and j.islower()==False and j!='\n':
                        flag=1
                        lines_uppercase.append([count+1,i])
                        print('Line no. {}:\n{}'.format(count+1,i.strip()))
    print('\n--------------------------------\n')

### CTE PADDING

In [4]:
def cte_padding(lines):
    lines_with_cte = []
    for count, i in enumerate(lines):
        if i.__contains__("with") or i.__contains__("as ("):
            lines_with_cte.append([count,i])
    reg = r'\s*\W\n'
    for i in lines_with_cte:
        if lines[i[0]+1].strip() != '':
            print("Padding absent after line no {} line - {}".format(i[0]+1,lines[i[0]]))
        if lines[i[0]-1].strip() != '':
            print("Padding absent before line no {} line - {}".format(i[0]+1,lines[i[0]]))
        if lines[i[0]+1].lstrip() == '' and lines[i[0]+2].lstrip() == '':
            print("Extra padding present after line no {} line - {}".format(i[0]+1,lines[i[0]]))
        if lines[i[0]-1].strip() == '' and lines[i[0]-2].strip() == '':
            print("Extra padding present before line no {} line - {}".format(i[0]+1,lines[i[0]]))
    lines_select = []
    for count, i in enumerate(lines):
        if i.__contains__("select") and not i.__contains__("select * from"):
            lines_select.append([count,i])
    reg = r'\s*\W\n'
    for i in lines_select:
        if lines[i[0]+1].strip() == '':
            print("Padding present after line no {} line - {}".format(i[0]+1,lines[i[0]]))
        if lines[i[0]-1].strip() == '' and lines[i[0]-2].strip() == '':
            print("Extra padding present before line no {} line - {}".format(i[0]+1,lines[i[0]]))

### CTE Alignment

In [11]:
def cte_alignment(text):
    def find_parens(s):
        toret = {}
        pstack = []

        for i, c in enumerate(s):
            if c == '(':
                pstack.append(i)
            elif c == ')':
                toret[pstack.pop()] = i
        return toret
    bracket_dict = find_parens(text)
    opening_bracks = list(bracket_dict.keys())
    closing_bracks = list(bracket_dict.values())
    cte_dict = {}
    for i in bracket_dict.keys():
        if text[i-4:i] == ' as ':
            cte_dict[i] = bracket_dict[i]
    for start_brack in cte_dict.keys():
        k = start_brack
        while text[k] != '\n':
            k=k-1
        cte_name = text[k:start_brack]
        if cte_name[1] == ' ':
            new_line_count = 0
            for i in text[:start_brack]:
                if i=='\n':
                    new_line_count+=1
            print('CTE Alignment incorrect for on line no. {}'.format(new_line_count+1))
        end_brack = cte_dict[start_brack]
        if text[end_brack-1] != '\n':
            new_line_count = 0
            for i in text[:end_brack]:
                if i=='\n':
                    new_line_count+=1
            print('CTE Alignment incorrect for on line no. {}'.format(new_line_count+1))


In [8]:
hundred_chars(lines)
comma_at_end(lines)
inner_join_keyword(lines)
not_equals_sign(lines)
snake_casing(lines)
cte_padding(lines)
cte_alignment(text)

Lines of SQL should be no longer than ~100 characters. Please use new line.

Line no. 70
qualify dense_rank() over ( partition by coalesce(salesforce_account_id, legacy_mindbody_sfdc_account_id, legacy_booker_sfdc_account_id)

Line no. 88
on dim_mindbody_legacy.legacy_mindbody_sfdc_account_id = core_software_subscription.legacy_mindbody_sfdc_account_id

Line no. 107
on dim_booker_legacy.legacy_booker_sfdc_account_id = core_software_subscription.legacy_booker_sfdc_account_id

Line no. 119
iff(core_software_subscription.platform='Mindbody', coalesce(to_varchar(dim_customer_mindbody_map.aria_id), dim_customer_mindbody_map.zuora_id), null)

Line no. 166
using dim_customer_mindbody_map.legacy_mindbody_sfdc_account_id = dim_mindbody_legacy.legacy_mindbody_sfdc_account_id

Line no. 186
on dim_customer_mindbody_map.legacy_mindbody_sfdc_account_id = mindbody_legacy.legacy_mindbody_sfdc_account_id

Line no. 193
{{dbt_utils.surrogate_key(['platform','coalesce(mindbody_studio_id::varchar,product_i

In [12]:
7 rules - combined

CTE Alignment incorrect for on line no. 19
CTE Alignment incorrect for on line no. 21
CTE Alignment incorrect for on line no. 30
CTE Alignment incorrect for on line no. 32
CTE Alignment incorrect for on line no. 41
